## Covid 19 Analysis on 3 different routes!

 - Covid 19 Cumulative data till 8th June
 - Statewise testing Details to look on the current scenario
 - Final Individual Details of **Current Status,Notes,City/District by date!**

In [110]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly as pt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go

In [111]:
covid_19_india = pd.read_csv('datasets_557629_1221351_covid_19_india.csv')
statewise_testing_details = pd.read_csv('datasets_557629_1221351_StatewiseTestingDetails.csv')
individual_details = pd.read_csv('IndividualDetails.csv')
PATH = (r'/Users/at20017363/Desktop/Miscellaneous Task/Covid19 Dataset kaggle')

In [112]:
# let's print the head for each of the DataFrames and then let's lay out a plan on which we can work upon!
covid_19_india.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,30/01/20,6:00 PM,Kerala,1,0,0,0,1
1,2,31/01/20,6:00 PM,Kerala,1,0,0,0,1
2,3,01/02/20,6:00 PM,Kerala,2,0,0,0,2
3,4,02/02/20,6:00 PM,Kerala,3,0,0,0,3
4,5,03/02/20,6:00 PM,Kerala,3,0,0,0,3


In [113]:
statewise_testing_details.head()

,Date,State,TotalSamples,Negative,Positive
0,2020-04-17,Andaman and Nicobar Islands,1403.0,1210.0,12.0
1,2020-04-24,Andaman and Nicobar Islands,2679.0,NaN,27.0
2,2020-04-27,Andaman and Nicobar Islands,2848.0,NaN,33.0
3,2020-05-01,Andaman and Nicobar Islands,3754.0,NaN,33.0
4,2020-05-16,Andaman and Nicobar Islands,6677.0,NaN,33.0


In [114]:
individual_details.head()

,id,government_id,diagnosed_date,age,gender,detected_city,detected_district,detected_state,nationality,current_status,status_change_date,notes
0,0,KL-TS-P1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,India,Recovered,14/02/2020,Travelled from Wuhan
1,1,KL-AL-P1,02/02/2020,NaN,NaN,Alappuzha,Alappuzha,Kerala,India,Recovered,14/02/2020,Travelled from Wuhan
2,2,KL-KS-P1,03/02/2020,NaN,NaN,Kasaragod,Kasaragod,Kerala,India,Recovered,14/02/2020,Travelled from Wuhan
3,3,DL-P1,02/03/2020,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,India,Recovered,15/03/2020,"Travelled from Austria, Italy"
4,4,TS-P1,02/03/2020,24,M,Hyderabad,Hyderabad,Telangana,India,Recovered,02/03/2020,"Travelled from Dubai to Bangalore on 20th Feb,..."


### So we attack on the below plan:

  - EDA for each of the **Data Sets** involving comparisons at different levels and stages.
  - Since each of the data consists of **Date** column, and hence we can look for **Time Series** modelling as there wouldn't be **seasonality** for sure, but there has to be a **trend**.
  - We keep on explaining the Steps on the way at each cell.
  
Let's Begin!!!

#### So the Data Analysis/Manipulation can be attacked in the below format!:

   1. Look at the shape,unique values and missing values for the columns!. Let's fix that we would drop the column having more than 40% of missing values. If at some point, that column is much needed, we would use the MODE/MEDIAN method for imputation based on the column's nature.
   2. In the COVID_19_INDIA dataset, there are multiple duplicates, because on the same day for multiple time stamps, multiple cases were recorded, so we have to tackle that in order to find the total RECOVERED/DECEASED......etc.
   3. Then we would like to plot the RECOVERED/DIAGNOSED/DETETCTED/DECEASED against time, to find the TREND.
   4. Find out which of the STATES are leading for different stages of the pandemic.

In [115]:
covid_19_india.head(15)

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,30/01/20,6:00 PM,Kerala,1,0,0,0,1
1,2,31/01/20,6:00 PM,Kerala,1,0,0,0,1
2,3,01/02/20,6:00 PM,Kerala,2,0,0,0,2
3,4,02/02/20,6:00 PM,Kerala,3,0,0,0,3
4,5,03/02/20,6:00 PM,Kerala,3,0,0,0,3
5,6,04/02/20,6:00 PM,Kerala,3,0,0,0,3
6,7,05/02/20,6:00 PM,Kerala,3,0,0,0,3
7,8,06/02/20,6:00 PM,Kerala,3,0,0,0,3
8,9,07/02/20,6:00 PM,Kerala,3,0,0,0,3
9,10,08/02/20,6:00 PM,Kerala,3,0,0,0,3


In [116]:
covid_19_india.shape

(2774, 9)

In [117]:
covid_19_india.describe(include='all')

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
count,2774.000000,2774,2774,2774,2774,2774,2774.000000,2774.000000,2774.000000
unique,NaN,130,7,37,62,12,NaN,NaN,NaN
top,NaN,30/05/20,8:00 AM,Kerala,-,-,NaN,NaN,NaN
freq,NaN,36,1136,130,2328,2328,NaN,NaN,NaN
mean,1387.500000,NaN,NaN,NaN,NaN,NaN,700.856525,52.670151,1764.340663
std,800.929148,NaN,NaN,NaN,NaN,NaN,2525.886014,212.795903,6094.283019
min,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
25%,694.250000,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,10.000000
50%,1387.500000,NaN,NaN,NaN,NaN,NaN,21.000000,1.000000,71.000000
75%,2080.750000,NaN,NaN,NaN,NaN,NaN,253.000000,15.000000,983.750000


In [118]:
# We see that CONFIREMD column is sum of Indian and Foreign Nationality, so let's use that. Instead of dropping those columns, we will subset the column.
# Let's find the latest date and for each state, let's pick the latest date entry.
# Also, i believe that this world is not much perfect, hence there will be definitely a spelling mistake in the States, let's check that first

covid_19_india['State/UnionTerritory'].value_counts()

Kerala                              130
Delhi                                98
Telengana                            98
Rajasthan                            97
Haryana                              96
Uttar Pradesh                        96
Ladakh                               93
Tamil Nadu                           93
Karnataka                            91
Maharashtra                          91
Jammu and Kashmir                    91
Punjab                               91
Andhra Pradesh                       88
Uttarakhand                          85
Odisha                               84
Puducherry                           82
West Bengal                          82
Chandigarh                           81
Chhattisgarh                         81
Gujarat                              80
Himachal Pradesh                     79
Madhya Pradesh                       79
Bihar                                78
Manipur                              76
Mizoram                              75


In [119]:
# Wow, the world is near perfection it seems so!😛.
# Ok, so we would like to club the UNASSGINED and CASES BEING REASSIGNED entries to some UNIDENTIFIED STATE, for better clarity.
covid_19_india['State/UnionTerritory'] = covid_19_india['State/UnionTerritory'].replace({'Unassigned': 'Unidentified state','Cases being reassigned to states':'Unidentified state'})
covid_19_india['State/UnionTerritory'].value_counts()

Kerala                         130
Delhi                           98
Telengana                       98
Rajasthan                       97
Haryana                         96
Uttar Pradesh                   96
Ladakh                          93
Tamil Nadu                      93
Karnataka                       91
Maharashtra                     91
Jammu and Kashmir               91
Punjab                          91
Andhra Pradesh                  88
Uttarakhand                     85
Odisha                          84
Puducherry                      82
West Bengal                     82
Chandigarh                      81
Chhattisgarh                    81
Gujarat                         80
Madhya Pradesh                  79
Himachal Pradesh                79
Bihar                           78
Manipur                         76
Mizoram                         75
Goa                             74
Andaman and Nicobar Islands     74
Assam                           68
Jharkhand           

In [120]:
# Now let's figure out the latest date for the DATE column
print('Last Recorded Date: ',covid_19_india['Date'].max())

Last Recorded Date:  31/05/20


In [121]:
# So we take this date and fetch the records for all the states
covid_19_by_state = covid_19_india[covid_19_india['Date']=='31/05/20']
covid_19_by_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 2486 to 2521
Data columns (total 9 columns):
Sno                         36 non-null int64
Date                        36 non-null object
Time                        36 non-null object
State/UnionTerritory        36 non-null object
ConfirmedIndianNational     36 non-null object
ConfirmedForeignNational    36 non-null object
Cured                       36 non-null int64
Deaths                      36 non-null int64
Confirmed                   36 non-null int64
dtypes: int64(4), object(5)
memory usage: 2.8+ KB


In [122]:
covid_19_by_state

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
2486,2487,31/05/20,8:00 AM,Andaman and Nicobar Islands,-,-,33,0,33
2487,2488,31/05/20,8:00 AM,Andhra Pradesh,-,-,2289,60,3569
2488,2489,31/05/20,8:00 AM,Arunachal Pradesh,-,-,1,0,4
2489,2490,31/05/20,8:00 AM,Assam,-,-,163,4,1185
2490,2491,31/05/20,8:00 AM,Bihar,-,-,1618,20,3636
2491,2492,31/05/20,8:00 AM,Chandigarh,-,-,189,4,289
2492,2493,31/05/20,8:00 AM,Chhattisgarh,-,-,102,1,447
2493,2494,31/05/20,8:00 AM,Dadar Nagar Haveli,-,-,0,0,2
2494,2495,31/05/20,8:00 AM,Delhi,-,-,8075,416,18549
2495,2496,31/05/20,8:00 AM,Goa,-,-,41,0,70


In [123]:
# So we get the latest data as per the 31st may entry!. Now, we can drop the nationality columns and start some plotting on this dataset.
# This dataset is merely suitable for exploration as some kind of ML Technique doesn't seems to be suitable for a small restricted dataset!.

covid_19_by_state.drop(['Sno','ConfirmedIndianNational','ConfirmedForeignNational','Time','Date'],axis=1,inplace=True)
covid_19_by_state.reset_index(inplace=True)
covid_19_by_state.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 5 columns):
index                   36 non-null int64
State/UnionTerritory    36 non-null object
Cured                   36 non-null int64
Deaths                  36 non-null int64
Confirmed               36 non-null int64
dtypes: int64(4), object(1)
memory usage: 1.5+ KB


In [124]:
pivoted_df = pd.pivot_table(covid_19_by_state, values=['Cured', 'Deaths','Confirmed'],columns=['State/UnionTerritory']).reset_index()
pivoted_df.rename({'index':'Condition'},inplace=True,axis=1)
#df.drop('State/UnionTerritory',inplace=True,axis=1)
pivoted_df

State/UnionTerritory,Condition,Andaman and Nicobar Islands,Andhra Pradesh,Arunachal Pradesh,Assam,Bihar,Chandigarh,Chhattisgarh,Dadar Nagar Haveli,Delhi,...,Punjab,Rajasthan,Sikkim,Tamil Nadu,Telengana,Tripura,Unidentified state,Uttar Pradesh,Uttarakhand,West Bengal
0,Confirmed,33,3569,4,1185,3636,289,447,2,18549,...,2233,8617,1,21184,2499,268,5491,7445,749,5130
1,Cured,33,2289,1,163,1618,189,102,0,8075,...,1967,5739,0,12000,1412,172,0,4410,102,1970
2,Deaths,0,60,0,4,20,4,1,0,416,...,44,193,0,160,77,0,0,201,5,309


In [125]:
# Let's try only one Plotly Visualisation comparing the States for CURED/DEATHS/CONFIRMED cases.

fig = px.bar(covid_19_by_state, y="State/UnionTerritory", x="Confirmed", orientation='h')
fig.show()

In [126]:
columns=pivoted_df.columns[1:]

fig = go.Figure(data=[
    go.Bar(name='Confirmed Cases', x=columns, y=pivoted_df.loc[0].values[1:]),
    go.Bar(name='Cured Cases', x=columns, y=pivoted_df.loc[1].values[1:]),
    go.Bar(name='Deaths Occured', x=columns, y=pivoted_df.loc[2].values[1:])
])
# Change the bar mode
fig.update_layout(barmode='stack',width=950, height=600,title_text = 'Combined View for every State!')
fig.update_yaxes(type="log")
fig.update_yaxes(nticks=10)
fig.update_layout(
    hoverlabel=dict(
        bgcolor="white", 
        font_size=16, 
        font_family="Rockwell"
    )
)
fig.show()

In [127]:
states = pivoted_df.columns[1:]

fig = go.Figure()
fig.add_trace(go.Bar(
    x=states,
    y=pivoted_df.loc[0].values[1:],
    name='Confimred',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=states,
    y=pivoted_df.loc[1].values[1:],
    name='Cured',
    marker_color='lightsalmon'
))
fig.add_trace(go.Bar(
    x=states,
    y=pivoted_df.loc[0].values[1:],
    name='Dead',
    marker_color='black'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-75,width=950, height=600,title_text = 'Combined View for every State, by CONDITION wise!')
fig.update_yaxes(type="log")
fig.update_yaxes(nticks=15)
fig.update_layout(
    hoverlabel=dict(
        bgcolor="white", 
        font_size=16, 
        font_family="Rockwell"
    )
)
comparison_plot = PATH +'/comparison_plot'
pt.offline.plot(fig,filename = comparison_plot)
fig.show()

What do we infer from the above 2 plots?

  1. **Maharashtra** is the leading State in India in terms of obtaining **Confirmed** cases. Well not only that, the state also leads in **Cured** cases and unfortunately for **Death** cases. Maybe Thakrey government is looking for some sorcery to happen.  
  2. Well, the Delhi is not so far behind and there is a slight difference between Delhi and Gujrat for all situations!.  
  3. In the below cell, we actually calculated the % of death and cureness and found that:  
      a. Maharashtra ->
          - Curing Percentage: 43.09%
          - Death Percentage: 3.37%
      b. Delhi:
          - Curing Percentage: 43.53%
          - Death Percentage: 2.24%
      c. Gujrat:
          - Curing Percentage: 56.48%
          - Death Percentage: 6.16%

This percenatge figure tells us, that although **Maharashtra** is having a larger number of Cases, but the speed of recovery is highest in Gujrat and so is the speed of Deaths!.


In [128]:
# Now, what we can do is we can calculate the Curing%,Death% and compare that for the states. This would help us identify those states, which are actually doing well although having large covid cases
covid_19_by_state['Cured_Precent'] = round(100*(covid_19_by_state['Cured']/covid_19_by_state['Confirmed']),2)
covid_19_by_state['Death_Precent'] = round(100*(covid_19_by_state['Deaths']/covid_19_by_state['Confirmed']),2)
covid_19_by_state.reset_index(drop=True)
covid_19_by_state

,index,State/UnionTerritory,Cured,Deaths,Confirmed,Cured_Precent,Death_Precent
0,2486,Andaman and Nicobar Islands,33,0,33,100.00,0.00
1,2487,Andhra Pradesh,2289,60,3569,64.14,1.68
2,2488,Arunachal Pradesh,1,0,4,25.00,0.00
3,2489,Assam,163,4,1185,13.76,0.34
4,2490,Bihar,1618,20,3636,44.50,0.55
5,2491,Chandigarh,189,4,289,65.40,1.38
6,2492,Chhattisgarh,102,1,447,22.82,0.22
7,2493,Dadar Nagar Haveli,0,0,2,0.00,0.00
8,2494,Delhi,8075,416,18549,43.53,2.24
9,2495,Goa,41,0,70,58.57,0.00


In [129]:
pivoted_df_percent = pd.pivot_table(covid_19_by_state, values=['Cured_Precent', 'Death_Precent'],columns=['State/UnionTerritory']).reset_index()
pivoted_df_percent.rename({'index':'Condition'},inplace=True,axis=1)
#df.drop('State/UnionTerritory',inplace=True,axis=1)
pivoted_df_percent

State/UnionTerritory,Condition,Andaman and Nicobar Islands,Andhra Pradesh,Arunachal Pradesh,Assam,Bihar,Chandigarh,Chhattisgarh,Dadar Nagar Haveli,Delhi,...,Punjab,Rajasthan,Sikkim,Tamil Nadu,Telengana,Tripura,Unidentified state,Uttar Pradesh,Uttarakhand,West Bengal
0,Cured_Precent,100.0,64.14,25.0,13.76,44.50,65.40,22.82,0.0,43.53,...,88.09,66.60,0.0,56.65,56.50,64.18,0.0,59.23,13.62,38.40
1,Death_Precent,0.0,1.68,0.0,0.34,0.55,1.38,0.22,0.0,2.24,...,1.97,2.24,0.0,0.76,3.08,0.00,0.0,2.70,0.67,6.02


In [130]:
states = pivoted_df_percent.columns[1:]

fig = go.Figure()
fig.add_trace(go.Bar(
    x=states,
    y=pivoted_df_percent.loc[0].values[1:],
    name='Cured Percentage',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=states,
    y=pivoted_df_percent.loc[1].values[1:],
    name='Death percentage',
    marker_color='lightsalmon'
))


# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-75,width=950, height=600,title_text = 'Combined View for every State, by CONDITION wise!')
fig.update_yaxes(type="log")
fig.update_yaxes(nticks=20)
fig.update_layout(
    hoverlabel=dict(
        bgcolor="white", 
        font_size=16, 
        font_family="Rockwell"
    )
)
cured_vs_death_percent = PATH +'/cured_vs_death_percent'
pt.offline.plot(fig,filename = cured_vs_death_percent)
fig.show()

In [131]:
# Let's pickup the states which are having more than 7K confirmed cases and then let's plot in order to do a proper comparison -> My Dad asked the similar question!
cases_above_7k = covid_19_by_state.loc[covid_19_by_state['Confirmed'] >= 7000]
cases_above_7k

,index,State/UnionTerritory,Cured,Deaths,Confirmed,Cured_Precent,Death_Precent
8,2494,Delhi,8075,416,18549,43.53,2.24
10,2496,Gujarat,9230,1007,16343,56.48,6.16
18,2504,Madhya Pradesh,4444,343,7891,56.32,4.35
19,2505,Maharashtra,28081,2197,65168,43.09,3.37
27,2513,Rajasthan,5739,193,8617,66.60,2.24
29,2515,Tamil Nadu,12000,160,21184,56.65,0.76
33,2519,Uttar Pradesh,4410,201,7445,59.23,2.70


In [132]:
cases_above_7k_pivot = pd.pivot_table(cases_above_7k, values=['Cured_Precent', 'Death_Precent'],columns=['State/UnionTerritory']).reset_index()
cases_above_7k_pivot.rename({'index':'Condition'},inplace=True,axis=1)
#df.drop('State/UnionTerritory',inplace=True,axis=1)
cases_above_7k_pivot

State/UnionTerritory,Condition,Delhi,Gujarat,Madhya Pradesh,Maharashtra,Rajasthan,Tamil Nadu,Uttar Pradesh
0,Cured_Precent,43.53,56.48,56.32,43.09,66.60,56.65,59.23
1,Death_Precent,2.24,6.16,4.35,3.37,2.24,0.76,2.70


In [133]:
states = cases_above_7k_pivot.columns[1:]

fig = go.Figure()
fig.add_trace(go.Bar(
    x=states,
    y=cases_above_7k_pivot.loc[0].values[1:],
    name='Cured Percentage',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=states,
    y=cases_above_7k_pivot.loc[1].values[1:],
    name='Death percentage',
    marker_color='lightsalmon'
))


# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-75,width=950, height=600,title_text = 'Combined View for every State, by CONDITION wise for states having more than 7k Confirmed Cases!')
fig.update_yaxes(type="log")
fig.update_yaxes(nticks=20)
fig.update_layout(
    hoverlabel=dict(
        bgcolor="white", 
        font_size=16, 
        font_family="Rockwell"
    )
)
fig.show()

Awesome, we did some bar Plotting till now, but i wanna see how does a pie chart looks for **Cured% and Death%** for the scenario of Confirmed Cases greater than 7K.

In [134]:
fig = px.pie(cases_above_7k, values='Cured_Precent', names='State/UnionTerritory')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide',title_text = 'Contribution of Curing percentage to 100% by the top States having more than 7k cases!')
curing_plot = PATH +'/curing_percent_plot'
pt.offline.plot(fig,filename = curing_plot)
fig.show()

In [135]:
labels = cases_above_7k['State/UnionTerritory']
values = cases_above_7k['Death_Precent']

# pull is given as a fraction of the pie radius
fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0.1, 0, 0])])
fig.update_layout(title_text = 'Contribution of Death percentage to 100% by the top States having more than 7k cases!')
death_percent_plot = PATH +'/death_percent_plot'
pt.offline.plot(fig,filename = death_percent_plot)
fig.show()

That's all for EDA -> Why?

We have already done 2 Kaggle Kernel:
  - [Covid Part 1](https://www.kaggle.com/mishra5001/till-now-what-corona-has-done-to-us)
  - [Covid Part 2](https://www.kaggle.com/mishra5001/extension-to-covid-19-analysis-part-2)
  
Hence, i'll stop doing EDA now, and work on some kind of ***Machine Learning Implemnentation*** on the Data Set of **individual details's**.

Stay Tuned!